Zillow API from:\
https://rapidapi.com/s.mahmoud97/api/zillow56

In [145]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

pd.options.display.max_columns = None

In [146]:
# Load API key
df = pd.read_csv('../../Data Analytics and Data Science Projects/API-KEYS.csv')
rapid_API_Key = df.loc[df['API'] == 'rapid-API']['KEY'].iloc[0]

In [147]:
# API specific variables.
url_search = "https://zillow56.p.rapidapi.com/search"
url_property = "https://zillow56.p.rapidapi.com/property"
headers = {
		"X-RapidAPI-Key":rapid_API_Key,
		"X-RapidAPI-Host":"zillow56.p.rapidapi.com"
	}

VALID_STATUS = {"forSale", "recentlySold", "forRent"}

In [148]:
# Set Zillow search parameters
location = "Brevard County FL"
status = "forSale" # Options are "forSale" or "recentlySold"
# priceRange = [[1, 250000], [250001, 500000], [500001, 750000], [750001, 1000000], [1000001, 2000000], [2000001, 5000000], [5000001, 50000000]] #Limiting for trial/error. Will set to full range once confirmed working.
priceRange = [[250001 500000]]

In [159]:
# Build functions for cleanliness
def zillowAPISearch(url=url_search, headers=headers, priceRange=priceRange, location="Brevard County FL", status="forSale"):

	if status not in VALID_STATUS:
		raise ValueError("Error: Status must be one of %r." %VALID_STATUS)
	
	for i, price in enumerate(priceRange):
		# for page in range(1,21): #Limiting for trial/error. Will set to full range once confirmed working.
		for page in range(1,2):
			querystring = {
				"page":page,
				"location":location,
				"status":status,
				"sortSelection":"priced",
				"isSingleFamily":"true",
				"isMultiFamily":"false",
				"isApartment":"false",
				"isCondo":"false",
				"isManufactured":"false",
				"isTownhouse":"false",
				"isLotLand":"false",
				"price_min":price[0],
				"price_max":price[1]}
		
			response = requests.get(url, headers=headers, params=querystring)
			time.sleep(1.5)
			
			if response.status_code != 200:
				print(f'Last page is: {page-1}')
				break

			searchData = response.json()

			if i == 0:
				df_searchData = pd.json_normalize(searchData['results'])
			else:
				df_searchData = pd.concat([df_searchData, pd.json_normalize(searchData['results'])], axis=0, ignore_index=True)
	
	return df_searchData

def zillowPropertyDetails(zpid, url=url_property, headers=headers):
	querystring = {
		"zpid":zpid
	}

	response = requests.get(url, headers=headers, params=querystring)
	time.sleep(1.5)

	if response.status_code != 200:
		raise ValueError('Chosen ZPID is not valid.')

	df_propertyData = pd.json_normalize(response.json())

	return df_propertyData

In [ ]:
# zillowAPISearch(url=url, headers=headers, location="Brevard County FL", status="forSale", priceRange=priceRange)

In [151]:
# querystring = {
# 		"zpid":"2053929120"
# 	}

# response = requests.get(url_property, headers=headers, params=querystring)

In [158]:
# df_propertyData = pd.json_normalize(response.json())
# df_propertyData.head()

,abbreviatedAddress,bathrooms,bedrooms,big,boroughId,boroughSearchUrl,brokerId,brokerIdDimension,brokerageName,building,buildingId,city,cityId,comingSoonOnMarketDate,comps,contingentListingType,country,county,countyFIPS,countyId,currency,datePostedString,datePriceChanged,dateSold,dateSoldString,daysOnZillow,description,desktopWebHdpImageLink,editPropertyHistorylink,enhancedBrokerImageUrl,favoriteCount,featuredListingTypeDimension,floorMaps,foreclosingBank,foreclosureAmount,foreclosureAuctionCity,foreclosureAuctionDescription,foreclosureAuctionFilingDate,foreclosureAuctionLocation,foreclosureAuctionTime,foreclosureBalanceReportingDate,foreclosureDate,foreclosureDefaultFilingDate,foreclosureJudicialType,foreclosureLoanAmount,foreclosureLoanDate,foreclosureLoanOriginator,foreclosureMoreInfo,foreclosurePastDueBalance,foreclosurePriorSaleAmount,foreclosurePriorSaleDate,foreclosureUnpaidBalance,hasApprovedThirdPartyVirtualTourUrl,hasBadGeocode,hasPublicVideo,hasVRModel,hdpTypeDimension,hdpUrl,hiResImageLink,hideZMA,hideZestimate,homeInsights,homeStatus,homeType,homeValues,hugePhotos,isCamo,isCommunityPillar,isCurrentSignedInAgentResponsible,isCurrentSignedInUserVerifiedOwner,isFeatured,isIncomeRestricted,isListedByOwner,isListingClaimedByCurrentSignedInUser,isNonOwnerOccupied,isPaidMultiFamilyBrokerId,isPreforeclosureAuction,isPremierBuilder,isRecentStatusChange,isRentalListingOffMarket,isRentalsLeadCapMet,isUndisclosedAddress,isZillowOwned,keystoneHomeStatus,lastSoldPrice,latitude,listPriceLow,listingAccountUserId,listingDataSource,listingFeedID,listingTypeDimension,livingArea,livingAreaUnits,livingAreaUnitsShort,livingAreaValue,longitude,lotAreaUnits,lotAreaValue,lotSize,mapTileGoogleMapUrlFullWidthLightbox,mapTileGoogleMapUrlFullWidthLightboxWholeZipcode,mapTileGoogleMapUrlFullWidthMax,mapTileGoogleMapUrlFullWidthMaxWholeZipcode,mapTileGoogleMapUrlSmall,mapTileGoogleMapUrlSmallWholeZipcode,marketingName,mediumImageLink,mlsid,monthlyHoaFee,mortgageRates,moveHomeMapLocationLink,moveInCompletionDate,moveInReady,nearbyCities,nearbyHomes,nearbyNeighborhoods,nearbyZipcodes,neighborhoodRegion,neighborhoodSearchUrl,newConstructionType,openHouseSchedule,pageUrlFragment,pageViewCount,pals,parcelId,photoCount,photos,postingProductType,postingUrl,price,priceChange,priceChangeDate,priceChangeDateString,priceHistory,primaryPublicVideo,propertyEventLogLink,propertyTaxRate,propertyTypeDimension,propertyUpdatePageLink,providerListingID,regionString,rentZestimate,rentalApplicationsAcceptedType,responsivePhotos,responsivePhotosOriginalRatio,restimateHighPercent,restimateLowPercent,richMedia,richMediaVideos,schools,sellingSoon,solarPotential,ssid,state,stateId,streetAddress,streetViewMetadataUrlMapLightboxAddress,streetViewMetadataUrlMediaWallAddress,streetViewMetadataUrlMediaWallLatLong,streetViewServiceUrl,streetViewTileImageUrlMediumAddress,streetViewTileImageUrlMediumLatLong,taxAssessedValue,taxAssessedYear,taxHistory,thumb,timeOnZillow,timeZone,tourPhotos,tourViewCount,unassistedShowing,virtualTourUrl,whatILove,yearBuilt,zestimate,zestimateHighPercent,zestimateLowPercent,zillowOfferMarket,zipPlusFour,zipcode,zoContactSubtitle,zoMarketName,zoResaleStartAnOfferEnabled,zpid,address.city,address.community,address.neighborhood,address.state,address.streetAddress,address.subdivision,address.zipcode,apartmentsForRentInZipcodeSearchUrl.path,attributionInfo.agentEmail,attributionInfo.agentLicenseNumber,attributionInfo.agentName,attributionInfo.agentPhoneNumber,attributionInfo.attributionTitle,attributionInfo.brokerName,attributionInfo.brokerPhoneNumber,attributionInfo.buyerAgentMemberStateLicense,attributionInfo.buyerAgentName,attributionInfo.buyerBrokerageName,attributionInfo.coAgentLicenseNumber,attributionInfo.coAgentName,attributionInfo.coAgentNumber,attributionInfo.infoList1,attributionInfo.infoList2,attributionInfo.infoString1,attributionInfo.infoString10,attributionInfo.infoString11,attributionInfo.infoString12,attributionInfo.infoString13,attrib

In [ ]:
# for page in range(1,21):
# 	url = "https://zillow56.p.rapidapi.com/search"

# 	querystring = {
# 		"location":"Brevard County FL",
# 		"page":page,
# 		"status":"forSale",
# 		"listing_type":"by_agent",
# 		"isSingleFamily":"true",
# 		"isMultiFamily":"false",
# 		"isApartment":"false",
# 		"isCondo":"false",
# 		"isManufactured":"false",
# 		"isTownhouse":"false",
# 		"isLotLand":"false"
# 	}

# 	headers = {
# 		"X-RapidAPI-Key":rapid_API_Key,
# 		"X-RapidAPI-Host":"zillow56.p.rapidapi.com"
# 	}

# 	response = requests.get(url, headers=headers, params=querystring)
# 	time.sleep(1.5)

# 	homeData = response.json()

# 	if page == 1:
# 		df_HomeData = pd.json_normalize(homeData['results'])
# 	else:
# 		df_HomeData = pd.concat([df_HomeData, pd.json_normalize(homeData['results'])], axis=0, ignore_index=True)


In [ ]:
# API calls for web scraping
# CAUTION: These will go against the monthly allowable fo 15,000. RUN SPARINGLY!!.
df_search = zillowAPISearch()

for i, zpid in enumerate(df['zpid']):
    if i == 0:
        df_property = zillowPropertyDetails(zpid=zpid)
    else:
        df_property = pd.concat([df_property, zillowPropertyDetails(zpid)], axis=0, ignore_index=True)

In [ ]:
# Concatenate the two dataframes together
df = pd.concat([df_search, df_property], axis=1, ignore_index=True)

# Export dataframe to csv file for later use.
df.to_csv('housingData.csv', sep=',', index=True, encoding='utf-8')